
# Tarea: Análisis con **pandas-on-Spark** usando `WorldCupPlayers.csv`

**Objetivo:** Replicar la **misma lógica** del notebook de referencia (Koalas) pero usando el dataset `WorldCupPlayers.csv` y la API moderna `pyspark.pandas` (pandas-on-Spark).

**Secciones a completar:**
1. Funciones básicas
2. Visualización de datos
3. Selección
4. Aplicando funciones de Python a pandas-on-Spark
5. Agrupando datos
6. Generando gráficos
7. Utilizando SQL en pandas-on-Spark
8. Trabajando con PySpark

> **Nota:** Este cuaderno incluye una celda de **arranque estable** para Spark en Windows/Anaconda y evita los problemas más comunes (PyArrow, timezone, Python worker).


In [1]:

# === Arranque estable para Spark en Windows/Anaconda ===
import os, sys

# 1) Fuerza el mismo intérprete Python en driver y ejecutores
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

# 2) Evita problemas de timestamps con Arrow
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

# 3) Enlaza el driver a localhost para evitar conflictos con VPN/firewall
os.environ["SPARK_LOCAL_IP"] = "127.0.0.1"

In [2]:

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("WorldCupPlayers-Tarea")
    .config("spark.executorEnv.PYSPARK_PYTHON", sys.executable)
    .config("spark.executorEnv.PYARROW_IGNORE_TIMEZONE", "1")
    .config("spark.driver.bindAddress", "127.0.0.1")
    .config("spark.driver.host", "127.0.0.1")
    .config("spark.sql.session.timeZone", "UTC")
    .getOrCreate()
)

import numpy as np
import pandas as pd
import pyspark.pandas as ps  # pandas-on-Spark (antes Koalas)

print("Spark Version:", spark.version)

Spark Version: 3.5.6



## Carga del dataset

Usaremos el archivo `WorldCupPlayers.csv` (ya disponible en este entorno).  
**Tareas:**
- Cargar el CSV con `ps.read_csv`.
- Inspeccionar el esquema y las primeras filas.
- Asegurar tipos adecuados (por ejemplo, convertir `Shirt Number` a numérico).


In [4]:

# Ruta local (ya cargado por el profesor en este entorno)
csv_path = "data/WorldCupPlayers.csv"

kdf = ps.read_csv(csv_path)
kdf.head()  # Vista rápida (dispara una pequeña recolección para representación)

c:\Users\osval\anaconda3\envs\ProcBigdata25B\lib\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,RoundID,MatchID,Team Initials,Coach Name,Line-up,Shirt Number,Player Name,Position,Event
0,201,1096,FRA,CAUDRON Raoul (FRA),S,0,Alex THEPOT,GK,None
1,201,1096,MEX,LUQUE Juan (MEX),S,0,Oscar BONFIGLIO,GK,None
2,201,1096,FRA,CAUDRON Raoul (FRA),S,0,Marcel LANGILLER,None,G40'
3,201,1096,MEX,LUQUE Juan (MEX),S,0,Juan CARRENO,None,G70'
4,201,1096,FRA,CAUDRON Raoul (FRA),S,0,Ernest LIBERATI,None,None



### 1) Funciones básicas

**Tareas:**  
- Crear una `Series` y un `DataFrame` simples con `ps.Series` / `ps.DataFrame`.  
- Seleccionar columnas (`["Player Name", "Team Initials", "Position"]`).  
- Contar valores nulos por columna en `kdf`.  
- Crear una nueva columna, por ejemplo `IsStarter = (Line-up == 'S')`.


In [5]:

# TODO: tu código aquí
# Ejemplos guía (puedes modificarlos):
ser = ps.Series([1, 3, 5, None, 8], name="demo")
demo_df = ps.DataFrame({"A":[1,2,3], "B":[10,20,30]})

subset = kdf[["Player Name", "Team Initials", "Position"]]
nulls = kdf.isna().sum()

kdf["IsStarter"] = (kdf["Line-up"] == "S")
subset.head(), nulls

(        Player Name Team Initials Position
 0       Alex THEPOT           FRA       GK
 1   Oscar BONFIGLIO           MEX       GK
 2  Marcel LANGILLER           FRA     None
 3      Juan CARRENO           MEX     None
 4   Ernest LIBERATI           FRA     None,
 RoundID              0
 MatchID              0
 Team Initials        0
 Coach Name           0
 Line-up              0
 Shirt Number         0
 Player Name          0
 Position         33641
 Event            28715
 dtype: int64)


### 2) Visualización de datos

**Tareas:**  
- Instalar/usar Plotly si es necesario.  
- Graficar **conteo de jugadores por `Position`** (Top 10).  
- Graficar **conteo de jugadores por `Team Initials`** (Top 10).

> Sugerencia: agrega `.to_pandas()` antes de pasar a Plotly para recolectar y graficar.


In [6]:

# TODO: tu código aquí
import plotly.express as px

pos_counts = (
    kdf["Position"]
    .value_counts()
    .reset_index()
    .rename(columns={"index":"Position","Position":"Count"})
    .to_pandas()
    .head(10)
)

fig1 = px.bar(pos_counts, x="Position", y="Count", title="Top 10 posiciones por número de jugadores")
fig1.show()

team_counts = (
    kdf["Team Initials"].value_counts()
    .reset_index()
    .rename(columns={"index":"Team Initials","Team Initials":"Count"})
    .to_pandas()
    .head(10)
)

fig2 = px.bar(team_counts, x="Team Initials", y="Count", title="Top 10 equipos por número de jugadores")
fig2.show()

c:\Users\osval\anaconda3\envs\ProcBigdata25B\lib\site-packages\pyspark\pandas\base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(
c:\Users\osval\anaconda3\envs\ProcBigdata25B\lib\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


c:\Users\osval\anaconda3\envs\ProcBigdata25B\lib\site-packages\pyspark\pandas\base.py:1437: FutureWarning:

The resulting Series will have a fixed name of 'count' from 4.0.0.

c:\Users\osval\anaconda3\envs\ProcBigdata25B\lib\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning:

`to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.




### 3) Selección

**Tareas:**  
- Filtrar solo **porteros** (`Position == 'GK'`) que sean **titulares** (`IsStarter == True`).  
- Seleccionar columnas relevantes y ordenar por `Team Initials`, `Player Name`.  
- Contar cuántos porteros titulares hay por equipo.


In [7]:

# TODO: tu código aquí
gk_starters = kdf[(kdf["Position"] == "GK") & (kdf["IsStarter"])]
gk_starters_sel = gk_starters[["Team Initials", "Player Name", "Position"]].sort_values(["Team Initials","Player Name"])

gk_by_team = gk_starters.groupby("Team Initials").size().sort_values(ascending=False)
gk_starters_sel.head(), gk_by_team.head(10)

(      Team Initials    Player Name Position
 31390           ALG       CHAOUCHI       GK
 15853           ALG  Larbi EL HADI       GK
 32173           ALG         MBOLHI       GK
 32863           ALG         MBOLHI       GK
 13570           ALG   Mahdi CERBAH       GK,
 Team Initials
 BRA    101
 ARG     80
 ITA     61
 FRG     59
 ENG     57
 FRA     53
 URU     50
 NED     50
 MEX     48
 SWE     46
 dtype: int64)


### 4) Aplicando funciones de Python a pandas-on-Spark

**Tareas:**  
- Definir una función Python que **limpie** el texto de `Coach Name` (por ejemplo, dejar solo el nombre).  
- Aplicarla con `.apply()` sobre la columna y crear `Coach Clean`.


In [8]:

# TODO: tu código aquí
import re

def clean_coach(x: str) -> str:
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return None
    # ejemplo: "CAUDRON Raoul (FRA)" -> "CAUDRON Raoul"
    return re.sub(r"\s*\([^)]*\)\s*$", "", str(x)).strip()

kdf["Coach Clean"] = kdf["Coach Name"].apply(clean_coach)
kdf[["Coach Name","Coach Clean"]].head(10)

,Coach Name,Coach Clean
0,CAUDRON Raoul (FRA),CAUDRON Raoul
1,LUQUE Juan (MEX),LUQUE Juan
2,CAUDRON Raoul (FRA),CAUDRON Raoul
3,LUQUE Juan (MEX),LUQUE Juan
4,CAUDRON Raoul (FRA),CAUDRON Raoul
5,LUQUE Juan (MEX),LUQUE Juan
6,CAUDRON Raoul (FRA),CAUDRON Raoul
7,LUQUE Juan (MEX),LUQUE Juan
8,CAUDRON Raoul (FRA),CAUDRON Raoul
9,LUQUE Juan (MEX),LUQUE Juan



### 5) Agrupando datos

**Tareas:**  
- Calcular **jugadores por equipo y posición** (pivot o groupby).  
- Calcular **titulares por equipo** usando `IsStarter`.  
- (Opcional) Extraer del campo `Event` cuántos goles **(si aplica)** por jugador/equipo.


In [9]:

# TODO: tu código aquí
players_by_team_pos = (
    kdf.groupby(["Team Initials","Position"])
    .size()
    .rename("Count")
    .reset_index()
    .sort_values(["Count"], ascending=False)
)

starters_by_team = (
    kdf.groupby("Team Initials")["IsStarter"]
    .mean()  # proporción de titulares S sobre total de filas del equipo
    .sort_values(ascending=False)
)

players_by_team_pos.head(10), starters_by_team.head(10)

c:\Users\osval\anaconda3\envs\ProcBigdata25B\lib\site-packages\pyspark\pandas\groupby.py:649: FutureWarning:

Default value of `numeric_only` will be changed to `False` instead of `True` in 4.0.0.



(    Team Initials Position  Count
 11            BRA     None   2150
 38            ITA     None   1663
 202           ARG     None   1594
 100           ENG     None   1235
 234           FRG     None   1213
 21            FRA     None   1209
 210           ESP     None   1197
 84            MEX     None   1070
 135           NED     None   1058
 125           URU     None   1033,
 Team Initials
 CUB    0.733333
 INH    0.611111
 BOL    0.554622
 EGY    0.530120
 ROU    0.528604
 SLV    0.523810
 MAR    0.516245
 USA    0.513032
 TCH    0.512422
 SWE    0.510081
 Name: IsStarter, dtype: float64)


### 6) Generando gráficos

**Tareas:**  
- Gráfico de barras: **jugadores por equipo y posición** (elige Top N).  
- (Opcional) Gráfico de pastel o barras apiladas por `Position`.


In [10]:

# TODO: tu código aquí
top = players_by_team_pos.to_pandas().head(20)
fig = px.bar(top, x="Team Initials", y="Count", color="Position",
             title="Top equipos/posiciones por número de jugadores (Top 20 filas)")
fig.show()

c:\Users\osval\anaconda3\envs\ProcBigdata25B\lib\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning:

`to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.




### 7) Utilizando SQL en pandas-on-Spark

**Tareas:**  
- Crear un `DataFrame` auxiliar (por ejemplo, **por año** si hubiera una columna comparable; en este dataset no hay `year` explícito, pero puedes crear un derivado desde `Event` si contiene info de goles/minutos) o simplemente usar dos vistas del mismo `kdf` con alias.  
- Ejecutar una consulta `ks.sql` (alias `ps.sql`) con **placeholders** y **pasando los dataframes como argumentos nombrados** (evita `KeyError`).

> **Ejemplo de patrón:**  
```python
res = ps.sql("""
    SELECT ks.[Team Initials] as team, COUNT(*) as n
    FROM {kdf} ks
    WHERE ks.Position = 'GK'
    GROUP BY ks.[Team Initials]
    ORDER BY n DESC
""", kdf=kdf)
```


In [12]:
res = ps.sql("""
    SELECT ks.`Team Initials` AS team, COUNT(*) AS n
    FROM {kdf} ks
    WHERE ks.Position = 'GK' AND ks.IsStarter = TRUE
    GROUP BY ks.`Team Initials`
    ORDER BY n DESC
""", kdf=kdf)

res.head(10)



,team,n
0,BRA,101
1,ARG,80
2,ITA,61
3,FRG,59
4,ENG,57
5,FRA,53
6,URU,50
7,NED,50
8,MEX,48
9,SWE,46



### 8) Trabajando con PySpark

**Tareas:**  
- Convertir `kdf` a `sdf` (`to_spark()`), usar `select`, `where`, `groupBy`, `orderBy`.  
- Mostrar resultados con `show()`.  
- Convertir resultados a pandas para graficar con Plotly si lo deseas.


In [13]:

sdf = kdf.to_spark()

res_sdf = (
    sdf
    .where("Position = 'GK' AND IsStarter = TRUE")
    .groupBy("`Team Initials`")
    .count()
    .orderBy("count", ascending=False)
)

res_sdf.show(10)

c:\Users\osval\anaconda3\envs\ProcBigdata25B\lib\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning:

If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.



+-------------+-----+
|Team Initials|count|
+-------------+-----+
|          BRA|  101|
|          ARG|   80|
|          ITA|   61|
|          FRG|   59|
|          ENG|   57|
|          FRA|   53|
|          URU|   50|
|          NED|   50|
|          MEX|   48|
|          SWE|   46|
+-------------+-----+
only showing top 10 rows



**Entrega (sugerida):**
- Sube este notebook con tus celdas completadas.
- Exporta además a HTML o PDF.
- Incluye capturas de tus gráficos.

